In [34]:
import sys
import numpy as np
import math
import random
import json
import requests
import random
import numpy as np
import tensorflow as tf

import gym
sys.path.append('c://Users/omara/Desktop/VSCODE/DigitalSquad/gym-maze/')
import gym_maze
from gym_maze.envs.maze_manager import MazeManager
from riddle_solvers import *

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from collections import deque

In [8]:
# # DQN hyperparameters
# BUFFER_SIZE = 100000
# BATCH_SIZE = 32
# GAMMA = 0.99
# EPSILON = 1.0
# EPSILON_DECAY = 0.9999
# EPSILON_MIN = 0.01
# LEARNING_RATE = 0.001
# UPDATE_FREQ = 1000

# # Define the Q-network
# def create_q_network(input_shape, num_actions):
#     model = Sequential()
#     model.add(Dense(32, input_shape=input_shape, activation='relu'))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dense(num_actions, activation=None))
#     model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE))
#     return model

# # Replay buffer for experience replay
# class ReplayBuffer:
#     def __init__(self):
#         self.buffer = []
#         self.buffer_size = BUFFER_SIZE
    
#     def add(self, experience):
#         if len(self.buffer) >= self.buffer_size:
#             self.buffer.pop(0)
#         self.buffer.append(experience)
    
#     def sample(self, batch_size):
#         if len(self.buffer) < batch_size:
#             return None
#         samples = random.sample(self.buffer, batch_size)
#         states, actions, rewards, next_states, dones = zip(*samples)
#         return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

# # DQN agent
# class DQNAgent:
#     def __init__(self, state_shape, num_actions):
#         self.q_network = create_q_network(state_shape, num_actions)
#         self.target_network = create_q_network(state_shape, num_actions)
#         self.buffer = ReplayBuffer()
#         self.num_actions = num_actions
#         self.step_count = 0
    
#     def select_action(self, state, epsilon):
#         if np.random.random() < epsilon:
#             action = np.random.choice(self.num_actions)
#         else:
#             q_values = self.q_network.predict(np.array([state]))[0]
#             action = np.argmax(q_values)
#         return action
    
#     def update_networks(self):
#         weights = self.q_network.get_weights()
#         self.target_network.set_weights(weights)
    
#     def train(self, batch_size):
#         if len(self.buffer.buffer) < batch_size:
#             return
#         states, actions, rewards, next_states, dones = self.buffer.sample(batch_size)
#         q_values_next = self.target_network.predict(next_states)
#         targets = rewards + (1 - dones) * GAMMA * np.amax(q_values_next, axis=1)
#         q_values = self.q_network.predict(states)
#         q_values[np.arange(batch_size), actions] = targets
#         self.q_network.fit(states, q_values, verbose=0)
    
#     def remember(self, state, action, reward, next_state, done):
#         self.buffer.add((state, action, reward, next_state, done))
    
#     def decay_epsilon(self):
#         epsilon = max(EPSILON_MIN, EPSILON_DECAY * self.step_count)
#         self.step_count += 1
#         return epsilon
    
#     def save_model(self, filename):
#         self.q_network.save(filename)
    
#     def load_model(self, filename):
#         self.q_network = tf.keras.models.load_model(filename)
#         self.target_network = create_q_network(self.q_network.input_shape[1:], self.num_actions)
#         self.update_networks()

# # Train the agent
# def train_agent(env, agent, num_episodes, batch_size):
#     scores = []
#     for episode in range(num_episodes):
#         state = env.reset()
#         done = False
#         score = 0
#         while not done:
#             action = agent.select_action(state, agent.decay_epsilon())
#             next_state, reward, done, info = env.step(action)
#             agent.remember(state, action, reward, next_state, done)
#             agent.train(batch_size)
#             state = next_state
#             score += reward
#         scores.append(score)
#         if episode % UPDATE_FREQ == 0:
#             agent.update_networks()
#             print('Episode: {}, Score: {}'.format(episode, score))
#     return scores

# # Test the agent
# def test_agent(env, agent, num_episodes):
#     scores = []
#     for episode in range(num_episodes):
#         state = env.reset()
#         done = False
#         score = 0
#         while not done:
#             action = agent.select_action(state, 0)
#             next_state, reward, done, info = env.step(action)
#             state = next_state
#             score += reward
#         scores.append(score)
#     return scores

# # Create the environment
# env = gym.make('maze-sample-10x10-v0')
# env = MazeManager(env)

# # Create the agent
# agent = DQNAgent(env.observation_space.shape, env.action_space.n)

# # Train the agent
# scores = train_agent(env, agent, 10000, BATCH_SIZE)

# # Test the agent
# test_scores = test_agent(env, agent, 100)

# # Save the model
# agent.save_model('dqn_maze.h5')

# # Plot the scores
# import matplotlib.pyplot as plt
# plt.plot(scores)
# plt.plot(test_scores)
# plt.show()





In [43]:
# DQN hyperparameters
BUFFER_SIZE = 100000
BATCH_SIZE = 32
GAMMA = 0.99
EPSILON = 1.0
EPSILON_DECAY = 0.9999
EPSILON_MIN = 0.01
LEARNING_RATE = 0.001
UPDATE_FREQ = 1000

# Define the Q-network
class QNetwork(nn.Module):
    def __init__(self, input_shape, num_actions):
        super().__init__()
        self.fc1 = nn.Linear(input_shape[0], 32)
        self.fc2 = nn.Linear(32, 32)
        self.fc3 = nn.Linear(32, num_actions)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Replay buffer for experience replay
class ReplayBuffer:
    
    def __len__(self):
        return len(self.buffer)
    
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.buffer = deque(maxlen=buffer_size)

    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):
        samples = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*samples)
        states = np.array(states)
        next_states = np.array(next_states)
        actions = np.array(actions)
        rewards = np.array(rewards)
        dones = np.array(dones)
        return states, actions, rewards, next_states, dones

# DQN agent
class DQNAgent:
    def __init__(self, state_shape, num_actions, BUFFER_SIZE):
        self.q_network = QNetwork(state_shape, num_actions)
        self.target_network = QNetwork(state_shape, num_actions)
        self.buffer = ReplayBuffer(buffer_size=BUFFER_SIZE)
        self.num_actions = num_actions
        self.step_count = 0
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=LEARNING_RATE)
    
    def select_action(self, state, epsilon):
        if np.random.random() < epsilon:
            action = np.random.choice(self.num_actions)
        else:
            q_values = self.q_network(torch.FloatTensor([state])).detach().numpy()[0]
            action = np.argmax(q_values)
        return action
    
    def update_networks(self):
        self.target_network.load_state_dict(self.q_network.state_dict())
    
    def train(self, buffer, batch_size):
        # Sample a batch of transitions from replay buffer
        states, actions, rewards, next_states, dones = buffer.sample(batch_size)

        # Convert arrays to tensors
        states = torch.tensor(states, dtype=torch.float32)
        actions = torch.tensor(actions, dtype=torch.int64)
        rewards = torch.tensor(rewards, dtype=torch.float32)
        next_states = torch.tensor(next_states, dtype=torch.float32)
        dones = torch.tensor(dones, dtype=torch.bool)

        # Calculate TD targets
        with torch.no_grad():
            q_next = self.target_network(next_states)
            q_next[dones] = 0.0
            targets = rewards + self.gamma * torch.max(q_next, dim=1)[0]

        # Calculate current Q-values and loss
        q_values = self.policy_network(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        loss = self.loss_fn(q_values, targets)

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Update target network
        self.soft_update_target_network(self.target_network, self.policy_network, self.tau)

    
    def remember(self, state, action, reward, next_state, done):
        self.buffer.add((state, action, reward, next_state, done))
    
    def decay_epsilon(self):
        epsilon = max(EPSILON_MIN, EPSILON_DECAY * self.step_count)
        self.step_count += 1
        return epsilon
    
    def save_model(self, filename):
        torch.save(self.q_network.state_dict(), filename)
    
    def load_model(self, filename):
        self.q_network.load_state_dict(torch.load(filename))
        self.target_network.load_state_dict(self.q_network.state_dict())
        

In [13]:
def manhattan_distance(start, end):
    return abs(start[0] - end[0]) + abs(start[1] - end[1])

def get_reward(obv):
    
        
    return 0 - manhattan_distance(obv[0], np.array((1, 1)))


In [44]:
# Initialize maze manager and environment
sample_maze = np.load("hackathon_sample.npy")
agent_id = "9" # add your agent id here
manager = MazeManager()
manager.init_maze(agent_id, maze_cells=sample_maze)
env = manager.maze_map[agent_id]

# Initialize DQN agent and replay buffer
state_shape = env.observation_space.shape
num_actions = env.action_space.n
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dqn_agent = DQNAgent(state_shape, num_actions, BUFFER_SIZE=10000)
memory = dqn_agent.buffer

# Training loop
EPSILON = 1.0
EPSILON_MIN = 0.01
EPSILON_DECAY = 0.995
BATCH_SIZE = 32
UPDATE_FREQ = 10
scores = []
for episode in range(500):
    state = env.reset()
    done = False
    score = 0
    while not done:
        # Choose action
        action = dqn_agent.select_action(state, EPSILON)

        # Take action
        observation, _, done, _, info = env.step(action)
        reward = get_reward(observation)

        score += reward

        # Store transition in replay buffer
        next_state = observation
        memory.add(state, action, reward, next_state, done)

        # Update state
        state = next_state

        # Train agent
        if len(memory) >= BATCH_SIZE:
            dqn_agent.train(memory, BATCH_SIZE)

        if done:
            break

    # Decay exploration rate
    EPSILON = max(EPSILON_MIN, EPSILON_DECAY * EPSILON)

    # Update target network
    if episode % UPDATE_FREQ == 0:
        dqn_agent.update_target_network()

    # Print episode info
    scores.append(score)
    print("Episode", episode, "completed with reward:", score)

# Save trained model
dqn_agent.save_model("trained_dqn.pt")

# Close environment
env.close()


0 rescue items


<ipython-input-43-377431d6d038>:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  states = np.array(states)
<ipython-input-43-377431d6d038>:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  next_states = np.array(next_states)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [15]:
# # Initialize maze manager and environment
# sample_maze = np.load("hackathon_sample.npy")
# agent_id = "9" # add your agent id here
# manager = MazeManager()
# manager.init_maze(agent_id, maze_cells=sample_maze)
# env = manager.maze_map[agent_id]

# # Initialize DQN agent and replay buffer
# state_shape = env.observation_space.shape
# num_actions = env.action_space.n
# dqn_agent = DQNAgent(state_shape, num_actions)
# memory = dqn_agent.buffer

# # Training loop
# scores = []
# for episode in range(500):
#     state = env.reset()
#     done = False
#     score = 0
#     while not done:
#         # Choose action
#         action = dqn_agent.select_action(state, EPSILON)

#         # Take action
#         observation, _, done, _, info = env.step(action)
#         reward = get_reward(observation)

#         score += reward

#         # Store transition in replay buffer
#         dqn_agent.remember(state, action, reward, next_state, done)

#         # Update state
#         state = next_state

#         # Train agent
#         dqn_agent.train(BATCH_SIZE)

#         if done:
#             break

#     # Decay exploration rate
#     EPSILON = max(EPSILON_MIN, EPSILON_DECAY * EPSILON)

#     # Update target network
#     if episode % UPDATE_FREQ == 0:
#         dqn_agent.update_networks()

#     # Print episode info
#     scores.append(score)
#     print("Episode", episode, "completed with reward:", score)

# # Save trained model
# dqn_agent.save_model("trained_dqn.h5")

# # Close environment
# env.close()


c:\Users\omara\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
c:\Users\omara\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
c:\Users\omara\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:195: UserWarning: WARN: The result returned by `env.reset()` was not a tuple of the form `(obs, info)`, where `obs` is a observation and `info` is a dictionary containing additional information. Actual type: `<class 'list'>`
  logger.warn(


0 rescue items


c:\Users\omara\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:137: UserWarning: WARN: The obs returned by the `step()` method was expecting a numpy array, actual type: <class 'list'>
  logger.warn(
c:\Users\omara\anaconda3\lib\site-packages\gym\spaces\box.py:227: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")
c:\Users\omara\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
c:\Users\omara\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:252: UserWarning: WARN: The reward returned by `step()` must be a float, int, np.integer or np.floating, actual type: <class 'NoneType'>
  logger.warn(
<ipython-input-8-f3a8a62f9190>:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:

# Test trained model
state = env.reset()
state = np.reshape(state, [1, state_size])
done = False
total_reward = 0
while not done:
    # Choose action
    action = dqn_agent.act(state, explore=False)

    # Take action
    next_state, reward, done, info = env.step(action)
    total_reward += reward

    # Update state
    state = np.reshape(next_state, [1, state_size])

    # Render environment
    env.render()

print("Test completed with reward:", total_reward)

# Close environment
env.close()

In [1]:
# import random
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from collections import deque

# class DQNAgent:
#     def __init__(self, state_size, action_size, seed):
#         self.state_size = state_size
#         self.action_size = action_size
#         self.seed = seed
        
#         # Hyperparameters
#         self.buffer_size = int(1e5)
#         self.batch_size = 64
#         self.gamma = 0.99
#         self.tau = 1e-3
#         self.lr = 5e-4
#         self.update_every = 4
#         self.learn_step = 0
        
#         # Q-Network
#         self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
#         self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
#         self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=self.lr)
        
#         # Replay memory
#         self.memory = ReplayBuffer(action_size, self.buffer_size, self.batch_size, seed)
        
#     def step(self, state, action, reward, next_state, done):
#         self.memory.add(state, action, reward, next_state, done)
        
#         self.learn_step += 1
#         if self.learn_step % self.update_every == 0:
#             if len(self.memory) > self.batch_size:
#                 experiences = self.memory.sample()
#                 self.learn(experiences, self.gamma)
                
#     def act(self, state, eps=0.):
#         state = torch.from_numpy(state).float().unsqueeze(0).to(device)
#         self.qnetwork_local.eval()
#         with torch.no_grad():
#             action_values = self.qnetwork_local(state)
#         self.qnetwork_local.train()
        
#         if random.random() > eps:
#             return np.argmax(action_values.cpu().data.numpy())
#         else:
#             return random.choice(np.arange(self.action_size))
        
#     def learn(self, experiences, gamma):
#         states, actions, rewards, next_states, dones = experiences
        
#         q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
#         q_targets = rewards + (gamma * q_targets_next * (1 - dones))
#         q_expected = self.qnetwork_local(states).gather(1, actions)
        
#         loss = F.mse_loss(q_expected, q_targets)
#         self.optimizer.zero_grad()
#         loss.backward()
#         self.optimizer.step()
        
#         self.soft_update(self.qnetwork_local, self.qnetwork_target, self.tau)
        
#     def soft_update(self, local_model, target_model, tau):
#         for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
#             target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)
            

# class QNetwork(nn.Module):
#     def __init__(self, state_size, action_size, seed):
#         super(QNetwork, self).__init__()
#         self.seed = torch.manual_seed(seed)
#         self.fc1 = nn.Linear(state_size, 64)
#         self.fc2 = nn.Linear(64, 64)
#         self.fc3 = nn.Linear(64, action_size)
        
#     def forward(self, state):
#         x = F.relu(self.fc1(state))
#         x = F.relu(self.fc2(x))
#         return self.fc3(x)
    
    
# class ReplayBuffer:
#     def __init__(self, action_size, buffer_size, batch_size, seed):
#         self.action_size = action_size
#         self.memory


In [ ]:
# import random
# import numpy as np
# import torch
# import torch.nn as nn
# import torch.optim as optim

# # Step 1: Initialize environment
# maze = Maze(width=10, height=10, complexity=0.2, density=0.2)
# env = MazeEnv(maze)

# # Step 2: Define DQN model architecture and create DQN agent
# class DQN(nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super(DQN, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 64)
#         self.fc2 = nn.Linear(64, 64)
#         self.fc3 = nn.Linear(64, output_dim)
    
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# class DQNAgent:
#     def __init__(self, state_dim, action_dim, gamma=0.99, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.999, lr=0.001, batch_size=64, memory_size=10000):
#         self.state_dim = state_dim
#         self.action_dim = action_dim
#         self.gamma = gamma
#         self.epsilon = epsilon
#         self.epsilon_min = epsilon_min
#         self.epsilon_decay = epsilon_decay
#         self.lr = lr
#         self.batch_size = batch_size
#         self.memory = []
#         self.memory_size = memory_size
#         self.model = DQN(state_dim, action_dim)
#         self.target_model = DQN(state_dim, action_dim)
#         self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
#         self.loss_fn = nn.MSELoss()
    
#     def act(self, state):
#         if random.random() < self.epsilon:
#             return random.randrange(self.action_dim)
#         else:
#             with torch.no_grad():
#                 q_values = self.model(torch.tensor(state).float())
#                 return torch.argmax(q_values).item()
    
#     def remember(self, state, action, reward, next_state, done):
#         self.memory.append((state, action, reward, next_state, done))
#         if len(self.memory) > self.memory_size:
#             self.memory.pop(0)
    
#     def replay(self):
#         if len(self.memory) < self.batch_size:
#             return
        
#         batch = random.sample(self.memory, self.batch_size)
#         states, actions, rewards, next_states, dones = zip(*batch)
#         states = torch.tensor(states).float()
#         actions = torch.tensor(actions).long()
#         rewards = torch.tensor(rewards).float()
#         next_states = torch.tensor(next_states).float()
#         dones = torch.tensor(dones).float()
        
#         q_values = self.model(states)
#         next_q_values = self.target_model(next_states).detach()
#         target_q_values = rewards + (1 - dones) * self.gamma * torch.max(next_q_values, dim=1)[0]
#         target_q_values = target_q_values.unsqueeze(1)
#         target
